<a href="https://colab.research.google.com/github/ashutosh-gupta2111/Exploratory_project/blob/master/Exploratory_Project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to the code of Exploratory Project of Ashutosh Gupta.**

In [ ]:
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2

Numpy is the library for helping in linear algebra.
Pandas is used for data visualization.
os is used mainly in giving file support.
cv2 is main python library that is used in this model for prediction. 


In [ ]:
labels = os.listdir('/content/grive/MyDrive/train')

In [ ]:
print(labels)

['no_yawn', 'Open', 'yawn', 'Closed']
